In [9]:
### Merge

from pymongo import MongoClient
from collections import Counter

In [20]:
db_names = ['orphanet','DO','ctdbase', 'disgenet', 'mesh','omim']
all_ids = set()
for db_name in db_names:
    db = MongoClient().mydisease[db_name]
    all_ids.update(set([x['_id'] for x in db.find({},{'_id':1})]))

In [21]:
Counter([x.split(":",1)[0] for x in all_ids])

Counter({'DOID': 6971,
         'MESH': 6821,
         'OMIM': 24891,
         'ObsoleteClass': 1,
         'Orphanet': 13104})